In [16]:
import aocd
from aocd.models import Puzzle
from aocd import submit
from collections import defaultdict
import itertools

In [2]:
current_day = 23
current_year = 2022
puzzle = Puzzle(year=current_year, day=current_day)
puzzle

<Puzzle(2022, 23) at 0x7fa54500ceb0 - Unstable Diffusion>

In [3]:
test_input = '''....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..'''

# test_input = '''.....
# ..##.
# ..#..
# .....
# ..##.
# .....'''

In [4]:
def get_input_data(test = 0):
    if test:
        input_data = test_input
    else:
        input_data = puzzle.input_data
    input_data = input_data.splitlines()
    return input_data

In [5]:
def neighborhood(row,col):
    for drow in [-1,0,1]:
        for dcol in [-1,0,1]:
            if drow==0 and dcol==0:
                continue
            else:
                yield (row+drow,col+dcol)

In [6]:
def west_neighbors(row,col):
    yield from [(row-1,col-1),(row,col-1),(row+1,col-1)]
    
def east_neighbors(row,col):
    yield from [(row-1,col+1),(row,col+1),(row+1,col+1)]

def south_neighbors(row,col):
    yield from [(row+1,col-1),(row+1,col),(row+1,col+1)]
    
def north_neighbors(row,col):
    yield from [(row-1,col-1),(row-1,col),(row-1,col+1)]

In [7]:
def crange(start, num_steps, modulo):
    '''
    circular range - starting at 'start', going for num_steps iterations, and output is modulo w.r.t. 'modulo'
    '''
    index = start % modulo
    for i in range(num_steps):
        yield index
        index = (index + 1) % modulo

In [8]:
def draw(locations):
    max_row = max(key[0] for key in locations)
    max_col = max(key[1] for key in locations)
    min_row = min(key[0] for key in locations)
    min_col = min(key[1] for key in locations)
    for row in range(min_row, max_row+1):
        print(''.join('#' if (row,col) in locations else '.'
                      for col in range(min_col, max_col+1)))
            

In [9]:
def count_empty_tiles(locations):
    max_row = max(key[0] for key in locations)
    max_col = max(key[1] for key in locations)
    min_row = min(key[0] for key in locations)
    min_col = min(key[1] for key in locations)
    return sum((row,col) not in locations 
               for row in range(min_row, max_row+1) 
               for col in range(min_col, max_col+1))
            

In [10]:
def count_empty_tiles_fast(locations):
    max_row = max(key[0] for key in locations)
    max_col = max(key[1] for key in locations)
    min_row = min(key[0] for key in locations)
    min_col = min(key[1] for key in locations)
    return (max_row-min_row+1) * (max_col-min_col+1) - len(locations)
            

In [21]:
# data = get_input_data(test=1)
data = get_input_data(test=0)

elf_id = 0
elf_locations, new_elf_locations = {}, {}
for row, line in enumerate(data):
    for col, char in enumerate(line):
        if char == '#':
            elf_locations[(row,col)] = elf_id
            elf_id += 1
num_elves = elf_id

rules = [north_neighbors, south_neighbors, west_neighbors, east_neighbors]
# directions = ['north','south','west','east']
directions = [(-1,0), (+1,0), (0,-1), (0,+1)]
num_rules = len(rules)
rule_start_idx = -1

elf_proposals = defaultdict(list)

# draw(elf_locations)
print(f"Start: # empty tiles = {count_empty_tiles_fast(elf_locations)}")

for round_num in itertools.count():
    # first half of the round - make proposals
    rule_start_idx = (rule_start_idx + 1) % num_rules
    num_need_to_move = 0
    elf_proposals.clear()
    for row,col in elf_locations:
        if any(neighbor in elf_locations for neighbor in neighborhood(row,col)):
            num_need_to_move += 1
            # all neighbors are not empty - have to propose a move
            for rule_id in crange(rule_start_idx, 
                                  num_steps=num_rules, 
                                  modulo=num_rules):
                nbr_func = rules[rule_id]
                if all(nbr not in elf_locations for nbr in nbr_func(row,col)):
                    drow, dcol = directions[rule_id]
                    # add this elf's id to the proposal for that (new_row, new_col) as destination
                    elf_proposals[(row+drow, col+dcol)].append((elf_locations[(row,col)],row,col))
                    break # out of rule_id for loop
            else: # if none of the conditions were met, stay put
                elf_proposals[(row,col)].append((elf_locations[(row,col)],row,col))
        else:
            # all neighbors are empty - stay put
            elf_proposals[(row,col)].append((elf_locations[(row,col)],row,col))

    if num_need_to_move == 0:
        print(f"Answer Part 2: No elves move in round {round_num+1}! Process ends")
        break
        
    # second half of the round - move to the proposed locations
    for row,col in elf_proposals:
        if len(elf_proposals[(row,col)]) == 1: # only one contender for this position - move there
            new_elf_locations[(row,col)] = elf_proposals[(row,col)][0][0]
        else:
            for idx,r,c in elf_proposals[(row,col)]:
                new_elf_locations[(r,c)] = idx

    # draw(new_elf_locations)
    assert num_elves == len(new_elf_locations)
    # swap dictionaries and clear the 2nd one
    elf_locations, new_elf_locations = new_elf_locations, elf_locations
    new_elf_locations.clear()
    if (round_num+1) == 10:
        print(f"Answer Part 1: Round {round_num+1}, # empty tiles = {count_empty_tiles_fast(elf_locations)}")
    elif (round_num+1) % 100 == 0:
        print(f"Round {round_num+1}, # empty tiles = {count_empty_tiles_fast(elf_locations)}")
        


Start: # empty tiles = 2512
Answer Part 1: Round 10, # empty tiles = 3871
Round 100, # empty tiles = 6591
Round 200, # empty tiles = 8601
Round 300, # empty tiles = 9681
Round 400, # empty tiles = 10926
Round 500, # empty tiles = 12112
Round 600, # empty tiles = 13347
Round 700, # empty tiles = 14501
Round 800, # empty tiles = 15159
Round 900, # empty tiles = 15695
Answer Part 2: No elves move in round 925! Process ends


## Part 1



In [12]:
puzzle.answer_a = 3871
puzzle.answer_a

'3871'

## Part 2



In [13]:
puzzle.answer_b = 925
puzzle.answer_b

'925'

In [14]:
%timeit count_empty_tiles(elf_locations)

1.48 ms ± 38.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit count_empty_tiles_fast(elf_locations)

323 µs ± 80.6 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
